# Interpret misorientation between tilts from matlab

In [ ]:
%matplotlib qt5
# All these are not needed
from pathlib import Path
import pickle

from orix import io, plot
# from orix.quaternion import Orientation
from orix.vector import Vector3d

import hyperspy.api as hs #General hyperspy package
import pyxem as pxm #Electron diffraction tools based on hyperspy
import numpy as np #General numerical and matrix support
import matplotlib.pyplot as plt #Plotting tools
import matplotlib.colors as mcolors #Some plotting color tools
from matplotlib.cm import ScalarMappable
import diffpy #Electron diffraction tools
import requests

import pandas as pd
# import seaborn as sb

#Import indexation and plotting tools
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from pyxem.utils import expt_utils as eutls

#Import orientation handling tools
from orix.quaternion import Rotation, symmetry, Orientation, Misorientation
from orix.vector.vector3d import Vector3d
from orix.projections import StereographicProjection
from orix import plot

from orix.crystal_map import Phase
from orix.vector import Miller
from diffpy.structure import Lattice, Structure



In [ ]:
def to_pandas(path):
    dfpath = Path(path)
    data = pd.read_csv(dfpath,sep="\s+",header = None,skiprows=1)
    data.columns = ["x","y","e1","e2","e3"]
    # data.index = [chr(i) for i in range(ord('a'),ord('z')+1)]
    # data['e1'] = np.rad2deg(data['e1'])
    # data['e2'] = np.rad2deg(data['e2'])
    # data['e3'] = np.rad2deg(data['e3'])
    return data

In [ ]:
# Import misorientation data from matlab script
tiltpath1 = Path('D:/Matlab_scripts/Matlab_ori_data2/tilt1p_oris_v3.txt')
tiltpath2 = Path('D:/Matlab_scripts/Matlab_ori_data2/tilt2p_oris_v3.txt')
tiltpath3 = Path('D:/Matlab_scripts/Matlab_ori_data2/tilt3p_oris_v3.txt')

In [ ]:
tilt1 = to_pandas(tiltpath1)
tilt2 = to_pandas(tiltpath2)
tilt3 = to_pandas(tiltpath3)

In [ ]:
# Calculate misorientation angle between two tilts
def misorientations(tilt_1,tilt_2):
    misors = np.zeros((len(tilt_1['e1'])))
    for i in range(len(tilt_1['e1'])):
        # euler_fundamental_region
        symm = symmetry.Oh
            # Specify two crystal structures and symmetries
        phase1 = Phase(
            point_group="m-3m",
            structure=Structure(lattice=Lattice(1, 1, 1, 90, 90, 90)),
        )
        phase2 = phase1
        ori1 = Orientation.from_euler([tilt_1['e1'][i],tilt_1['e2'][i],tilt_1['e3'][i]])
        ori2 = Orientation.from_euler([tilt_2['e1'][i],tilt_2['e2'][i],tilt_2['e3'][i]])        
        m = Misorientation(ori2 * (~ori1), symmetry = (symm, symm))

        misor =  np.rad2deg(m.map_into_symmetry_reduced_zone().angle) 
        misors[i] = misor
    df = pd.DataFrame({'x':tilt_1['x'],'y':tilt_1['y'],'Mis. angle':misors})
    return misors,df

In [ ]:
# Calculate misorientation angle and convert to dataframe
misors12, df12 = misorientations(tilt1,tilt2)
misors13, df13 = misorientations(tilt1,tilt3)
misors23, df23 = misorientations(tilt2,tilt3)

In [ ]:
# Plot histograms for misorientation angles of grains, between tilts
# Tilt12
plt.figure()
plt.title('Misorientation angle tilts 1 and 2')
plt.xlabel('Degrees [$\degree$]')
plt.ylabel('Number of grains')
plt.hist(misors12,bins=60)
plt.xticks([0,5,10,20,30,40,50,60])
plt.show()

# Tilt13
plt.figure()
plt.title('Misorientation angle tilts 1 and 3')
plt.xlabel('Degrees [$\degree$]')
plt.ylabel('Number of grains')
plt.hist(misors13,bins=60)
plt.xticks([0,7,10,20,30,40,50,60])
plt.show()

# Tilt23
plt.figure()
plt.title('Misorientation angle tilts 2 and 3')
plt.xlabel('Degrees [$\degree$]')
plt.ylabel('Number of grains')
plt.hist(misors23,bins=60)
plt.xticks([0,5,10,20,30,40,50,60])
plt.show()